In [1]:
# TensorFlow / Keras
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Audio Processing
import librosa
import librosa.display

# System & Utilities
import os
import numpy as np
import matplotlib.pyplot as plt

2025-11-03 20:58:32.511766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762203512.699981      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762203512.752835      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_path="/kaggle/input/audio-mfcc/MFCC/Train"
test_path="/kaggle/input/audio-mfcc/MFCC/Test"
val_path="/kaggle/input/audio-mfcc/MFCC/Validation"

In [3]:
train_datagen = ImageDataGenerator( 
                                   rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                  ) 
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(  rescale=1./255.0)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (299,299),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (299,299),
                                            batch_size = 32,
                                            class_mode = 'binary',
                                            shuffle=False)
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (299,299),
                                            batch_size = 32,
                                            class_mode = 'binary')
# data input shape
input_shape = (299, 299, 3) 

Found 13956 images belonging to 2 classes.
Found 1088 images belonging to 2 classes.
Found 2826 images belonging to 2 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Input shape and weights path
input_shape = (299, 299, 3)
weights_path = '/kaggle/input/xceptionet-weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Load Xception base model
base_model = Xception(weights=weights_path, include_top=False, input_shape=input_shape)

# Unfreeze last 40 layers
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-40:]:
    layer.trainable = True

# Reduced dropout and removed BatchNorm
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

combined_model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=1e-5)
combined_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)

# Train model
history = combined_model.fit(
    training_set,
    epochs=20,
    validation_data=val_set,
    callbacks=[early_stop, reduce_lr]
)

I0000 00:00:1762203540.564953      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1762203540.565602      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1762203561.123727      81 service.cc:148] XLA service 0x7c60a04165b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762203561.124472      81 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1762203561.124491      81 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1762203562.898745      81 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-11-03 20:59:28.269310: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng3{k11=0} for conv (f32[32,128,147,147]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,128,147,147]{3,2,1,0}, f32[128,128,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0

437/437 ━━━━━━━━━━━━━━━━━━━━ 488s 1s/step - accuracy: 0.5613 - loss: 0.6756 - val_accuracy: 0.7286 - val_loss: 0.5704 - learning_rate: 1.0000e-05
Epoch 2/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 360s 822ms/step - accuracy: 0.7733 - loss: 0.4922 - val_accuracy: 0.7155 - val_loss: 0.5267 - learning_rate: 1.0000e-05
Epoch 3/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 372s 850ms/step - accuracy: 0.8181 - loss: 0.4074 - val_accuracy: 0.8015 - val_loss: 0.4200 - learning_rate: 1.0000e-05
Epoch 4/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 374s 856ms/step - accuracy: 0.8426 - loss: 0.3572 - val_accuracy: 0.8493 - val_loss: 0.3653 - learning_rate: 1.0000e-05
Epoch 5/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 388s 887ms/step - accuracy: 0.8600 - loss: 0.3276 - val_accuracy: 0.8464 - val_loss: 0.3476 - learning_rate: 1.0000e-05
Epoch 6/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 365s 835ms/step - accuracy: 0.8703 - loss: 0.3110 - val_accuracy: 0.8719 - val_loss: 0.3011 - learning_rate: 1.0000e-05
Epoch 7/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 362s 827ms/ste

In [5]:
# Save the model
combined_model.save('/kaggle/working/Deepfake_Audio_Model(2).h5')

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

test_path="/kaggle/input/audio-mfcc/MFCC/Test"
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (299,299),
                                            batch_size = 32,
                                            class_mode = 'binary',
                                            shuffle=False)
# Load the model
model = load_model('/kaggle/working/Deepfake_Audio_Model(2).h5')

loss, accuracy = model.evaluate(test_set)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Found 1088 images belonging to 2 classes.
34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 381ms/step - accuracy: 0.7673 - loss: 0.5177
Test Loss: 0.7697054743766785
Test Accuracy: 0.6351103186607361


In [7]:
from sklearn.metrics import f1_score, classification_report

# Load the model
model = load_model('/kaggle/input/latest-model/keras/default/1/Deepfake_Audio_Model(2).h5')

# Predict on test data
y_pred_probs = model.predict(test_set)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()  # Threshold at 0.5 for binary classification

# True labels
y_true = test_set.classes

# F1 Score
f1 = f1_score(y_true, y_pred)
print("F1 Score:", f1)

# Optional: Detailed report
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=test_set.class_indices.keys()))


34/34 ━━━━━━━━━━━━━━━━━━━━ 13s 264ms/step
F1 Score: 0.4287515762925599

Classification Report:

              precision    recall  f1-score   support

        fake       0.55      0.85      0.67       544
        real       0.68      0.31      0.43       544

    accuracy                           0.58      1088
   macro avg       0.62      0.58      0.55      1088
weighted avg       0.62      0.58      0.55      1088



In [8]:
import os
import librosa
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# Function to convert audio to MFCC and save as an image for prediction
def audio_to_mfcc_image(audio_file_path, target_size=(299, 299)):
    # Load the audio file using librosa
    audio, sr = librosa.load(audio_file_path, sr=None)  # sr=None keeps the original sampling rate

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=audio, sr=sr)
    # Resize the MFCC to match the input size of the model (299x299)
    mfcc_resized = librosa.util.fix_length(mfcc, size=target_size[0])
    mfcc_resized = np.expand_dims(mfcc_resized, axis=-1)  # Add the channel dimension

    # Convert MFCC to image format
    mfcc_image = np.repeat(mfcc_resized, 3, axis=-1)  

    # Resize the MFCC image to the target size
    mfcc_image = np.resize(mfcc_image, (target_size[0], target_size[1], 3))

    return mfcc_image

# Load the trained model
#model = combined_model
model = load_model('/kaggle/input/latest-model/keras/default/1/Deepfake_Audio_Model(2).h5')

# Define the path to the audio file
audio_file_path = os.path.join('/kaggle/input/in-the-wild-audio-deepfake/release_in_the_wild/fake/10002.wav') 

# Convert audio file to MFCC image
mfcc_image = audio_to_mfcc_image(audio_file_path)

# Add an extra dimension to match the model input shape (batch dimension)
mfcc_image = np.expand_dims(mfcc_image, axis=0)

# Preprocess the MFCC image (same preprocessing used during training for Xception)
mfcc_image = preprocess_input(mfcc_image)

# Make a prediction
prediction = model.predict(mfcc_image)

# Interpret the result
predicted_class = 'Fake' if prediction[0][0] > 0.5 else 'Real'
print(f'The predicted class is: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
The predicted class is: Fake


In [9]:
import os
import numpy as np
import librosa
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import load_model

# ==========================================================
# Function to convert audio to MFCC image for prediction
# ==========================================================
def audio_to_mfcc_image(audio_file_path, target_size=(299, 299)):
    try:
        # Load the audio file using librosa
        audio, sr = librosa.load(audio_file_path, sr=None)

        # Extract MFCC features
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)

        # Fix MFCC length to match model input size
        mfcc_resized = librosa.util.fix_length(mfcc, size=target_size[0])

        # Expand dimensions and convert to RGB-like array
        mfcc_resized = np.expand_dims(mfcc_resized, axis=-1)
        mfcc_image = np.repeat(mfcc_resized, 3, axis=-1)
        mfcc_image = np.resize(mfcc_image, (target_size[0], target_size[1], 3))

        return mfcc_image

    except Exception as e:
        print(f"❌ Error processing audio file: {e}")
        return None


# ==========================================================
# Load the trained model
# ==========================================================
try:
    model_path = '/kaggle/input/latest-model/keras/default/1/Deepfake_Audio_Model(2).h5'
    model = load_model(model_path)
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    model = None


# ==========================================================
# Define your test audio file path here
# ==========================================================
# Example: Upload a file to your dataset and replace the path below
audio_file_path = '/kaggle/input/testing-data-real/New Recording 3.wav'
# or use your own uploaded file like:
# audio_file_path = '/kaggle/input/my-audio-folder/sample.wav'


# ==========================================================
# Convert audio to MFCC image and predict
# ==========================================================
if model and os.path.exists(audio_file_path):
    mfcc_image = audio_to_mfcc_image(audio_file_path)

    if mfcc_image is not None:
        # Add batch dimension
        mfcc_image = np.expand_dims(mfcc_image, axis=0)

        # Preprocess as required by Xception model
        mfcc_image = preprocess_input(mfcc_image)

        # Predict
        try:
            prediction = model.predict(mfcc_image)
            predicted_class = 'Fake' if prediction[0][0] > 0.5 else 'Real'
            confidence = prediction[0][0] if predicted_class == 'Fake' else 1 - prediction[0][0]

            print(f"\n🔍 Prediction: {predicted_class}")
            print(f"📊 Confidence: {confidence:.2f}")
        except Exception as e:
            print(f"❌ Error during prediction: {e}")
    else:
        print("⚠️ Could not convert audio to MFCC image.")
else:
    print("⚠️ Model not loaded or audio file not found.")

✅ Model loaded successfully.


/tmp/ipykernel_19/392065778.py:13: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

🔍 Prediction: Fake
📊 Confidence: 0.90


In [10]:
import numpy as np
unique, counts = np.unique(training_set.classes, return_counts=True)
print(dict(zip(training_set.class_indices.keys(), counts)))


{'fake': 6978, 'real': 6978}
